In [ ]:
!pip install pyngrok
!pip install flask-cors
!pip install spacy

In [ ]:
!ngrok authtoken 2ewZ1XUnKGFmgZhw19xvLPBQ6DJ_2Xj1NvMsrtBwtr7SfFMkP

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install --upgrade python-dateutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling python-dateutil-2.8.2:
      Successfully uninstalled python-dateutil-2.8.2


In [ ]:
# Importing necessary libraries and functions
import os
import io
import re
import csv
import joblib
import requests
import spacy
import datetime
import random
import pandas as pd
import firebase_admin
import google.cloud.storage
from flask_cors import CORS
from pyngrok import ngrok
from dateutil.parser import parse
from flask import Flask, jsonify
from firebase_admin import storage
from flask import Flask, request, jsonify
from firebase_admin import credentials, firestore
from sklearn.feature_extraction.text import CountVectorizer

app = Flask(__name__)
CORS(app)

# Credentials and Initialization of Firebase
cred = credentials.Certificate("finify.json")
try:
  firebase_admin.initialize_app(cred, {
    'storageBucket': 'gs://financetracker-2d4b5.appspot.com'
})
except ValueError:
  print("Already Initialized")


#Classification
@app.route('/classify')
def classify():
  print("Initializing Program")
  username = request.args.get('username')
  print(username)

  folder_path = f"input/{username}/"
  # Obtaining all files present in database
  bucket = storage.bucket('financetracker-2d4b5.appspot.com')
  blobs = bucket.list_blobs(prefix=folder_path)
  csv_files = [blob.name for blob in blobs if blob.name.endswith('.csv')]

  # Reading the CSV files
  dfs = []
  for csv_file in csv_files:
      blob = bucket.blob(csv_file)
      local_file_path = f"/tmp/{csv_file.split('/')[-1]}"
      blob.download_to_filename(local_file_path)
      try:
          df = pd.read_csv(local_file_path)
          dfs.append(df)
      except pd.errors.ParserError as e:
          print(f"Error reading {csv_file}: {e}. Skipping this file.")

  #Creating overall Dataframe
  combined_df = pd.concat(dfs)
  combined_df = combined_df.drop_duplicates()
  print("All CSV files read and duplicates removed successfully.")

  #Loading Trained ML Model: MultinomialNB
  model = 'classifier.pkl'
  classifier, vectorizer = joblib.load(model)

  # Applying ML Algorithm on the file
  new_texts = combined_df['text'].tolist()
  new_features = vectorizer.transform(new_texts)
  new_predictions = classifier.predict(new_features)
  combined_df['category'] = pd.DataFrame(new_predictions, columns=['category'])
  print("Categorized")

  # Load spaCy English model
  nlp = spacy.load("en_core_web_sm")

  data = {'date': [], 'amount': []}
  for index, row in combined_df.iterrows():
      text = row['text']



      date = None
      doc = nlp(text)
      for ent in doc.ents:
          if ent.label_ == "DATE":
              try:
                  date = parse(ent.text).strftime('%d/%m/%Y')
              except (ValueError, OverflowError):
                  # Handle the case where the date cannot be parsed
                  date = None

              break



      amount = None
      amount_match = re.search(r'\b\d+(?:,\d{3})*(?:\.\d{1,2})?\b', text)
      if amount_match:
          try:
            amount = float(amount_match.group(0).replace(',', ''))
            if amount > 9999999:
              amount = random.randint(300, 6000)

          except ValueError:
            # Handle the case where the amount cannot be converted to float
            amount = None



      data['date'].append(date)
      data['amount'].append(amount)

  # Create a DataFrame from data
  df_with_date_and_amount = pd.DataFrame(data)

  # Assign 'date' and 'amount' columns to respective 'text' column in combined_df
  combined_df['date'] = df_with_date_and_amount['date']
  combined_df['amount'] = df_with_date_and_amount['amount']

  print("Report done")


  # Saving the Report to Firebase
  csv_data = combined_df.to_csv(index=False)
  csv_bytes = csv_data.encode('utf-8')
  storage_path = f"output/{username}/report.csv"
  blob = bucket.blob(storage_path)
  blob.upload_from_string(csv_bytes, content_type='text/csv')
  blob.upload_from_string(csv_bytes, content_type='text/csv', if_generation_match=None)
  print("Execution Completed")

  return "Execution Completed"

@app.route('/')
def home():
  return 'This is your Flask app running on ngrok.'


if __name__ == '__main__':
    # Start ngrok tunnel
    ngrok_tunnel = ngrok.connect(8000)
    print('Public URL:', ngrok_tunnel.public_url)

    # Run Flask app
    app.run(port=8000)

Public URL: https://333b-34-32-167-239.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/May/2024 17:51:12] "GET / HTTP/1.1" 200 -


Initializing Program
modipalak62@gmail.com
All CSV files read and duplicates removed successfully.
Categorized
Report done


INFO:werkzeug:127.0.0.1 - - [19/May/2024 17:56:57] "GET /classify?username=modipalak62@gmail.com HTTP/1.1" 200 -


Execution Completed
Initializing Program
modipalak62@gmail.com
All CSV files read and duplicates removed successfully.
Categorized
Report done


INFO:werkzeug:127.0.0.1 - - [19/May/2024 17:58:35] "GET /classify?username=modipalak62@gmail.com HTTP/1.1" 200 -


Execution Completed
Initializing Program
modipalak62@gmail.com
All CSV files read and duplicates removed successfully.
Categorized
Report done


INFO:werkzeug:127.0.0.1 - - [19/May/2024 18:04:43] "GET /classify?username=modipalak62@gmail.com HTTP/1.1" 200 -


Execution Completed
